In [1]:
import os
import json
import pandas as pd
import re

# EDA

In [6]:
dir = '../data/raw/20231212/funds/'
symbol_list = []
category = []
asset_class = []
inception_date = []
min_inv = []
exp_ratio = []
return_ytd = []
return_1_year = []
return_3_year = []
return_5_year = []
return_10_year = []
return_inception = []
return_bench_ytd = []
return_bench_1_year = []
return_bench_3_year = []
return_bench_5_year = []
return_bench_10_year = []
return_bench_inception = []
risk_level = []


for filename in os.listdir(dir):
    file_path = os.path.join(dir,filename)
    with open(file_path,'r') as f:
        fund_dict = json.load(f)
    pattern = r'\/([A-Z]+)\.json'
    match = re.search(pattern, file_path)
    symbol = match.group(1)
    symbol_list.append(symbol)
    
    category.append(fund_dict['key_fact_table']['Category'])
    asset_class.append(fund_dict['key_fact_table']['Asset class'])
    if 'Inception date' in fund_dict['key_fact_table']:
        inception_date.append(fund_dict['key_fact_table']['Inception date'])
    else:
        inception_date.append(None)

    if 'risk_level' in fund_dict:
        risk_level.append(fund_dict['risk_level'])
    else:
        risk_level.append(None)

    if 'perf_table' in fund_dict:
        perf_table = fund_dict['perf_table']

        if symbol in perf_table:
            symbol_key = symbol
        elif symbol+'1' in perf_table:
            symbol_key = symbol + '1'
        elif symbol+'2' in perf_table:
            symbol_key = symbol+'2'
        elif symbol+'4' in perf_table:
            symbol_key = symbol+'4'
        else:
            symbol_key = symbol+' (Market price)'
        
        if 'Benchmark1' in perf_table:
            benchmark_key = 'Benchmark1'
        elif 'Benchmark3' in perf_table:
            benchmark_key = 'Benchmark3'
        else:
            benchmark_key = 'Benchmark'

        for row in zip(perf_table['index'],perf_table[symbol_key],perf_table[benchmark_key]):
            if row[0]=="YTD":
                return_ytd.append(row[1])
                return_bench_ytd.append(row[2])
            if row[0]=="1-yr":
                return_1_year.append(row[1])
                return_bench_1_year.append(row[2])
            if row[0]=="3-yr":
                return_3_year.append(row[1])
                return_bench_3_year.append(row[2])
            if row[0]=="5-yr":
                return_5_year.append(row[1])
                return_bench_5_year.append(row[2])
            if row[0]=="10-yr":
                return_10_year.append(row[1])
                return_bench_10_year.append(row[2])
            if row[0]=="Since inception":
                return_inception.append(row[1]) 
                return_bench_inception.append(row[2])
    else:
        return_ytd.append(None)
        return_bench_ytd.append(None)
        return_1_year.append(None)
        return_bench_1_year.append(None)
        return_3_year.append(None)
        return_bench_3_year.append(None)
        return_5_year.append(None)
        return_bench_5_year.append(None)
        return_10_year.append(None)
        return_bench_10_year.append(None)
        return_inception.append(None)
        return_bench_inception.append(None)

    if 'min_investment' in fund_dict:
        min_inv.append(fund_dict['min_investment'])
    else:
        min_inv.append(None)
    
    if 'exp_ratio' in fund_dict:
        exp_ratio.append(fund_dict['exp_ratio'])
    else:
        exp_ratio.append(None)    

In [7]:
table_col = ['symbol_list','category','asset_class','inception_date','min_inv','exp_ratio','return_ytd','return_1_year','return_3_year','return_5_year','return_10_year','return_inception','return_bench_ytd','return_bench_1_year','return_bench_3_year','return_bench_5_year','return_bench_10_year','return_bench_inception','risk_level']

fund_df = pd.DataFrame(columns=table_col,data=list(zip(symbol_list,category,asset_class,inception_date,min_inv,exp_ratio,return_ytd,return_1_year,return_3_year,return_5_year,return_10_year,return_inception,return_bench_ytd,return_bench_1_year,return_bench_3_year,return_bench_5_year,return_bench_10_year,return_bench_inception,risk_level)))

In [8]:
fund_df[fund_df['risk_level'].isna()]

,symbol_list,category,asset_class,inception_date,min_inv,exp_ratio,return_ytd,return_1_year,return_3_year,return_5_year,return_10_year,return_inception,return_bench_ytd,return_bench_1_year,return_bench_3_year,return_bench_5_year,return_bench_10_year,return_bench_inception,risk_level
26,VSMAX,Small Blend,Domestic Stock - More Aggressive,11/13/2000,"$3,000",0.05%,7.18%,0.81%,3.74%,6.99%,7.65%,8.59%,7.08%,0.71%,3.70%,6.95%,7.63%,8.54%,None
61,IVOO,Mid Blend,Domestic Stock - More Aggressive,09/07/2010,$1.00,0.10%,7.06%,1.08%,7.25%,8.03%,8.57%,11.26%,7.10%,1.17%,7.36%,8.12%,8.69%,11.39%,None
71,VDE,Equity Energy,Domestic Stock - Sector-Specific,09/23/2004,$1.00,0.10%,0.19%,-3.73%,38.95%,10.04%,2.96%,7.21%,0.26%,-3.67%,39.11%,10.14%,3.10%,7.21%,None
174,VYMI,Foreign Large Value,International/Global Stock,02/25/2016,$1.00,0.22%,11.50%,9.91%,8.00%,6.09%,—,7.40%,11.71%,11.84%,8.22%,6.35%,3.74%,7.79%,None
191,VO,Mid Blend,Domestic Stock - More Aggressive,01/26/2004,$1.00,0.04%,8.31%,2.51%,4.51%,8.90%,8.98%,9.21%,8.26%,2.47%,4.51%,8.90%,9.01%,9.24%,None
240,VIOO,Small Blend,Domestic Stock - More Aggressive,09/07/2010,$1.00,0.10%,2.90%,-4.00%,5.73%,5.59%,7.46%,11.33%,2.89%,-4.02%,5.85%,5.63%,7.51%,11.42%,None
245,VIOV,Small Value,Domestic Stock - More Aggressive,09/07/2010,$1.00,0.15%,1.30%,-5.36%,8.13%,5.64%,6.87%,10.66%,1.31%,-5.30%,8.30%,5.73%,6.99%,10.81%,None
253,BLV,Long-Term Bond,Long-Term Bond,04/03/2007,$1.00,0.04%,-0.17%,-2.53%,-11.03%,0.36%,2.41%,4.22%,-0.72%,-2.09%,-11.03%,0.32%,2.38%,4.21%,None
255,VB,Small Blend,Domestic Stock - More Aggressive,01/26/2004,$1.00,0.05%,7.26%,0.90%,3.75%,6.99%,7.66%,8.54%,7.08%,0.71%,3.70%,6.95%,7.63%,8.49%,None
262,VGAVX,Emerging Markets Bond,Taxable Bond,05/31/2013,"$3,000",0.20%,4.33%,4.39%,-4.60%,0.86%,2.47%,2.04%,5.19%,5.40%,-4.43%,0.96%,2.56%,2.14%,None


# Cleaning Helper Functions

In [ ]:
def perc_to_decimal(perc_string):
    return round(float((perc_string.strip().replace('%',''))/100),2)